In [1]:
!git clone -b ema https://github.com/edoppiap/bert_medical_records.git

Cloning into 'bert_medical_records'...
remote: Enumerating objects: 422, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 422 (delta 24), reused 38 (delta 16), pack-reused 353
Receiving objects: 100% (422/422), 306.28 KiB | 6.96 MiB/s, done.
Resolving deltas: 100% (251/251), done.


In [2]:
!pip install -r bert_medical_records/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.12.2
    Uninstalling fsspec-2023.12.2:
      Successfully uninstalled fsspec-2023.12.2
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.4
    Uninstalling pandas-2.1.4:
      Successfully 

# Create pre-train and finetuning dataset text files

In [3]:
!python /kaggle/working/bert_medical_records/preprocessing_python/text_generator.py \
        --file_path /kaggle/input/new-base-data/PHeP_simulated_data.csv \
        --output_folder /kaggle/working \
        --output_name dataset.txt \
        --create_pretrain_text_file

Producing text file from csv dataset: 100%|██| 680/680 [00:01<00:00, 595.57it/s]


In [4]:
!python /kaggle/working/bert_medical_records/preprocessing_python/text_generator.py \
        --file_path /kaggle/working/dataset.txt \
        --output_folder /kaggle/working \
        --output_name nsp_dataset.txt \
        --create_nsp_text_file

Creating nsp dataset: 10811it [00:00, 801935.14it/s]


In [5]:
!python /kaggle/working/bert_medical_records/preprocessing_python/text_generator.py \
        --file_path /kaggle/input/new-base-data/PHeP_simulated_data.csv \
        --output_folder /kaggle/working \
        --output_name finetuning_dataset.txt \
        --create_finetuning_text_data

Reading input file: 100%|████████████████████| 680/680 [00:01<00:00, 490.20it/s]
Creating dataset for finetuning: 11481it [00:00, 152100.78it/s]


# Both mlm_nsp train and eval

### Only our pre-train

In [6]:
!python bert_medical_records/run_pre_train.py \
      --do_eval \
      --do_train \
      --train_batch_size 10 \
      --pre_train_tasks mlm_nsp \
      --num_epochs 8 \
      --input_file /kaggle/working/nsp_dataset.txt \
      --output_dir /kaggle/working/output/our_pretrain_only_mlm_nsp 

2024-05-25 10:14:49.739905: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-25 10:14:49.740004: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-25 10:14:49.869942: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Output files will be saved in folder: /kaggle/working/output/our_pretrain_only_mlm_nsp
Train the tokenizer
[00:00:00] Tokenize words                 ██████████████████ 855      /      855
[00:00:00] Count pairs                    ██████████████████ 855      /      855
[00:00:00] Compute merges                 ██████████████████ 1243     /     1243
Saving the toke

In [7]:
!python /kaggle/working/bert_medical_records/run_glue.py \
      --do_train \
      --do_eval \
      --train_batch_size 10 \
      --num_epochs 1 \
      --model_input /kaggle/working/output/our_pretrain_only_mlm_nsp \
      --input_file /kaggle/working/finetuning_dataset.txt \
      --output_dir /kaggle/working/output/our_pretrain_only_mlm_nsp

2024-05-25 11:36:28.465995: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-25 11:36:28.466049: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-25 11:36:28.467557: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Output files will be saved in folder: /kaggle/working/output/our_pretrain_only_mlm_nsp
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /kaggle/working/output/our_pretrain_only_mlm_nsp/pre_trained_model and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-

### Bert pretrained without further pretrain

In [8]:
!python bert_medical_records/run_pre_train.py \
      --do_eval \
      --train_batch_size 10 \
      --pre_train_tasks mlm_nsp \
      --input_file /kaggle/working/nsp_dataset.txt \
      --use_pretrained_bert

2024-05-25 11:46:02.709980: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-25 11:46:02.710046: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-25 11:46:02.711630: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
tokenizer_config.json: 100%|██████████████████| 48.0/48.0 [00:00<00:00, 209kB/s]
vocab.txt: 100%|█████████████████████████████| 232k/232k [00:00<00:00, 11.5MB/s]
tokenizer.json: 100%|████████████████████████| 466k/466k [00:00<00:00, 7.33MB/s]
config.json: 100%|█████████████████████████████| 570/570 [00:00<00:00, 2.96MB/s]
model.safetensors: 100%|█████████████████

In [9]:
!python /kaggle/working/bert_medical_records/run_glue.py \
      --do_train \
      --do_eval \
      --train_batch_size 10 \
      --num_epochs 1 \
      --input_file /kaggle/working/finetuning_dataset.txt \
      --output_dir /kaggle/working/output/just_bert_mlm_nsp \
      --use_pretrained_bert

2024-05-25 11:49:48.233845: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-25 11:49:48.233921: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-25 11:49:48.235441: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Output files will be saved in folder: /kaggle/working/output/just_bert_mlm_nsp
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

### Bert pretrained with our pretrain

In [10]:
!python bert_medical_records/run_pre_train.py \
      --do_eval \
      --do_train \
      --train_batch_size 10 \
      --pre_train_tasks mlm_nsp \
      --num_epochs 8 \
      --input_file /kaggle/working/nsp_dataset.txt \
      --output_dir /kaggle/working/output/bert_mlm_nsp_with_pretrain \
      --use_pretrained_bert

2024-05-25 11:59:24.655845: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-25 11:59:24.655914: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-25 11:59:24.657458: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Output files will be saved in folder: /kaggle/working/output/bert_mlm_nsp_with_pretrain
  0%|                                                   | 0/865 [00:00<?, ?it/s]/kaggle/working/bert_medical_records/load_dataset.py:147: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a fut

In [11]:
!python /kaggle/working/bert_medical_records/run_glue.py \
      --do_train \
      --do_eval \
      --train_batch_size 10 \
      --num_epochs 1 \
      --model_input /kaggle/working/output/bert_mlm_nsp_with_pretrain \
      --input_file /kaggle/working/finetuning_dataset.txt \
      --output_dir /kaggle/working/output/bert_mlm_nsp_with_pretrain

2024-05-25 13:33:00.971215: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-25 13:33:00.971275: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-25 13:33:00.972930: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Output files will be saved in folder: /kaggle/working/output/bert_mlm_nsp_with_pretrain
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /kaggle/working/output/bert_mlm_nsp_with_pretrain/pre_trained_model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a dow

# Only mlm pretrain

### Only our pre-train

In [12]:
!python bert_medical_records/run_pre_train.py \
      --do_eval \
      --do_train \
      --train_batch_size 10 \
      --pre_train_tasks mlm \
      --num_epochs 8 \
      --input_file /kaggle/working/dataset.txt \
      --output_dir /kaggle/working/output/our_pretrain_only_mlm

2024-05-25 13:42:46.751754: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-25 13:42:46.751807: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-25 13:42:46.753244: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Output files will be saved in folder: /kaggle/working/output/our_pretrain_only_mlm
Train the tokenizer
[00:00:00] Tokenize words                 ██████████████████ 854      /      854
[00:00:00] Count pairs                    ██████████████████ 854      /      854
[00:00:00] Compute merges                 ██████████████████ 1248     /     1248
Saving the tokenize

In [13]:
!python /kaggle/working/bert_medical_records/run_glue.py \
      --do_train \
      --do_eval \
      --train_batch_size 10 \
      --num_epochs 1 \
      --model_input /kaggle/working/output/our_pretrain_only_mlm \
      --input_file /kaggle/working/finetuning_dataset.txt \
      --output_dir /kaggle/working/output/our_pretrain_only_mlm

2024-05-25 13:48:14.024125: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-25 13:48:14.024191: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-25 13:48:14.025848: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Output files will be saved in folder: /kaggle/working/output/our_pretrain_only_mlm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /kaggle/working/output/our_pretrain_only_mlm/pre_trained_model and are newly initialized: ['bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight', 'bert.pooler.dense.bias']
Y

### Bert pretrained without further pretrain

In [14]:
!python bert_medical_records/run_pre_train.py \
      --do_eval \
      --train_batch_size 10 \
      --pre_train_tasks mlm \
      --input_file /kaggle/working/dataset.txt \
      --use_pretrained_bert

2024-05-25 13:57:53.975318: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-25 13:57:53.975373: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-25 13:57:53.977022: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture 

In [15]:
!python /kaggle/working/bert_medical_records/run_glue.py \
      --do_train \
      --do_eval \
      --train_batch_size 10 \
      --num_epochs 1 \
      --input_file /kaggle/working/finetuning_dataset.txt \
      --output_dir /kaggle/working/output/just_bert_mlm_only \
      --use_pretrained_bert

2024-05-25 13:58:24.834083: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-25 13:58:24.834138: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-25 13:58:24.835546: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Output files will be saved in folder: /kaggle/working/output/just_bert_mlm_only
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inf

### Bert pretrained with our pretrain

In [16]:
!python bert_medical_records/run_pre_train.py \
      --do_eval \
      --do_train \
      --train_batch_size 10 \
      --pre_train_tasks mlm \
      --num_epochs 8 \
      --input_file /kaggle/working/dataset.txt \
      --output_dir /kaggle/working/output/bert_only_mlm_with_pretrain \
      --use_pretrained_bert

2024-05-25 14:08:06.119274: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-25 14:08:06.119337: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-25 14:08:06.120837: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Output files will be saved in folder: /kaggle/working/output/bert_only_mlm_with_pretrain
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMask

In [17]:
!python /kaggle/working/bert_medical_records/run_glue.py \
      --do_train \
      --do_eval \
      --train_batch_size 10 \
      --num_epochs 1 \
      --model_input /kaggle/working/output/bert_only_mlm_with_pretrain \
      --input_file /kaggle/working/finetuning_dataset.txt \
      --output_dir /kaggle/working/output/bert_only_mlm_with_pretrain

2024-05-25 14:14:27.868501: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-25 14:14:27.868555: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-25 14:14:27.869980: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Output files will be saved in folder: /kaggle/working/output/bert_only_mlm_with_pretrain
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /kaggle/working/output/bert_only_mlm_with_pretrain/pre_trained_model and are newly initialized: ['bert.pooler.dense.weight', 'classifier.weight', 'classifier.bias', 'bert.pooler.de

# Only NSP

### Only our pretrain

In [18]:
!python bert_medical_records/run_pre_train.py \
      --do_eval \
      --do_train \
      --train_batch_size 10 \
      --pre_train_tasks nsp \
      --num_epochs 8 \
      --input_file /kaggle/working/nsp_dataset.txt \
      --output_dir /kaggle/working/output/our_pretrain_only_nsp

2024-05-25 14:24:16.754513: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-25 14:24:16.754579: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-25 14:24:16.756090: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Output files will be saved in folder: /kaggle/working/output/our_pretrain_only_nsp
Train the tokenizer
[00:00:00] Tokenize words                 ██████████████████ 855      /      855
[00:00:00] Count pairs                    ██████████████████ 855      /      855
[00:00:00] Compute merges                 ██████████████████ 1243     /     1243
Saving the tokenize

In [19]:
!python /kaggle/working/bert_medical_records/run_glue.py \
      --do_train \
      --do_eval \
      --train_batch_size 10 \
      --num_epochs 1 \
      --model_input /kaggle/working/output/our_pretrain_only_nsp \
      --input_file /kaggle/working/finetuning_dataset.txt \
      --output_dir /kaggle/working/output/our_pretrain_only_nsp

2024-05-25 15:30:09.242577: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-25 15:30:09.242639: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-25 15:30:09.244096: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Output files will be saved in folder: /kaggle/working/output/our_pretrain_only_nsp
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /kaggle/working/output/our_pretrain_only_nsp/pre_trained_model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream t

### Bert pretrained with no further pretrain

In [20]:
!python bert_medical_records/run_pre_train.py \
      --do_eval \
      --train_batch_size 10 \
      --pre_train_tasks nsp \
      --input_file /kaggle/working/nsp_dataset.txt \
      --use_pretrained_bert

2024-05-25 15:39:55.459036: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-25 15:39:55.459093: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-25 15:39:55.460551: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Evaluating:   0%|                                       | 0/217 [00:00<?, ?it/s]/kaggle/working/bert_medical_records/load_dataset.py:147: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arang

In [21]:
!python /kaggle/working/bert_medical_records/run_glue.py \
      --do_train \
      --do_eval \
      --train_batch_size 10 \
      --num_epochs 1 \
      --input_file /kaggle/working/finetuning_dataset.txt \
      --output_dir /kaggle/working/output/just_bert_only_nsp \
      --use_pretrained_bert

2024-05-25 15:41:18.330989: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-25 15:41:18.331047: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-25 15:41:18.332530: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Output files will be saved in folder: /kaggle/working/output/just_bert_only_nsp
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inf

### Bert pretrained with our pretrain

In [22]:
!python bert_medical_records/run_pre_train.py \
      --do_eval \
      --do_train \
      --train_batch_size 10 \
      --pre_train_tasks nsp \
      --num_epochs 8 \
      --input_file /kaggle/working/nsp_dataset.txt \
      --output_dir /kaggle/working/output/bert_nsp_with_pretrain

2024-05-25 15:51:06.100553: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-25 15:51:06.100604: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-25 15:51:06.102131: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Output files will be saved in folder: /kaggle/working/output/bert_nsp_with_pretrain
Train the tokenizer
[00:00:00] Tokenize words                 ██████████████████ 855      /      855
[00:00:00] Count pairs                    ██████████████████ 855      /      855
[00:00:00] Compute merges                 ██████████████████ 1243     /     1243
Saving the tokeniz

In [23]:
!python /kaggle/working/bert_medical_records/run_glue.py \
      --do_train \
      --do_eval \
      --train_batch_size 10 \
      --num_epochs 1 \
      --model_input /kaggle/working/output/bert_nsp_with_pretrain \
      --input_file /kaggle/working/finetuning_dataset.txt \
      --output_dir /kaggle/working/output/bert_nsp_with_pretrain

2024-05-25 16:57:05.389773: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-25 16:57:05.389825: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-25 16:57:05.391371: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Output files will be saved in folder: /kaggle/working/output/bert_nsp_with_pretrain
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /kaggle/working/output/bert_nsp_with_pretrain/pre_trained_model and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream

# Create zip folder

In [24]:
import shutil

# Specify the path to the folder you want to zip
folder_path = '/kaggle/working/output'

# Specify the path for the resulting zip file
zip_file_path = '/kaggle/working/output'

# Create a zip file
shutil.make_archive(zip_file_path, 'zip', folder_path)

'/kaggle/working/output.zip'